Инициализация рабочей директории и настроек

In [1]:
import os
import sys
from pathlib import Path

def init_work_dir(path):
    # Установить рабочую директорию
    project_root = Path(path)
    os.chdir(project_root)
    # Добавить путь к модулям проекта
    sys.path.insert(0, str(project_root))
    print(f"Текущая рабочая директория: {os.getcwd()}")

init_work_dir("C:/Projects/GeoOffice")

from src.utils.file_utils import FileUtils
from src.models.settings_model import Settings

settings = Settings(data=None)
settings_data = FileUtils.load_json(Path("storage") / "data" / "settings.json")
try:
    settings = Settings(data=settings_data)
    print("✅ Настройки загружены успешно")
except Exception as e:
    print(f"Ошибка чтения файла настроек: {e}")
    print("Используются настройки по умолчанию")

DB_PATH = settings.paths.database_path
print(f"База даных: {DB_PATH}")

Текущая рабочая директория: C:\Projects\GeoOffice
09:57:41 | INFO     | GeoOffice | ============================================================
09:57:41 | INFO     | GeoOffice | 🚀 Запуск приложения GeoOffice
09:57:41 | INFO     | GeoOffice | 📅 Дата и время: 2025-07-07 09:57:41
09:57:41 | INFO     | GeoOffice | 🐍 Версия Python: 3.13.5 (tags/v3.13.5:6cb20a2, Jun 11 2025, 16:15:46) [MSC v.1943 64 bit (AMD64)]
09:57:41 | INFO     | GeoOffice | 📁 Рабочая директория: C:\Projects\GeoOffice
09:57:41 | INFO     | GeoOffice | 📂 Директория логов: C:\Projects\GeoOffice\storage\logs
09:57:41 | INFO     | GeoOffice | ============================================================
09:57:41 | DEBUG    | GeoOffice.utils.files | 📂 Загрузка JSON файла: storage\data\settings.json
09:57:41 | INFO     | GeoOffice.utils.files | ✅ JSON файл загружен: storage\data\settings.json
✅ Настройки загружены успешно
База даных: \\Server-r\igi\projects.db


Установка / снятие защиты

In [5]:
import subprocess

def show_db_file():
    subprocess.run(['attrib', '-h', '-s', str(DB_PATH)], check=True)
def protect_db_file():
    subprocess.run(['attrib', '+h', '+s', str(DB_PATH)], check=True)

# show_db_file()  # Снятие защиты
# protect_db_file()  # Установка защиты

Удаление БД

In [4]:
from pathlib import Path

def delete_db_file():
    Path(DB_PATH).unlink()
    if Path(DB_PATH).exists():
        raise Exception("Файл не удалён")

# delete_db_file()

FileNotFoundError: [WinError 2] Не удается найти указанный файл: '\\\\Server-r\\igi\\projects.db'

Создание / Подключение к БД

In [2]:
from src.services.database_service import DataBaseProjectService

db_service = DataBaseProjectService(DB_PATH)

20:41:19 | INFO     | GeoOffice | ============================================================
20:41:19 | INFO     | GeoOffice | 🚀 Запуск приложения GeoOffice
20:41:19 | INFO     | GeoOffice | 📅 Дата и время: 2025-07-05 20:41:19
20:41:19 | INFO     | GeoOffice | 🐍 Версия Python: 3.13.5 (tags/v3.13.5:6cb20a2, Jun 11 2025, 16:15:46) [MSC v.1943 64 bit (AMD64)]
20:41:19 | INFO     | GeoOffice | 📁 Рабочая директория: C:\Projects\GeoOffice
20:41:19 | INFO     | GeoOffice | 📂 Директория логов: C:\Projects\GeoOffice\storage\logs
20:41:19 | INFO     | GeoOffice | ============================================================
20:41:19 | INFO     | GeoOffice.services.database_service | Инициализация базы данных: C:\GeoOffice_test_server\projects.db
20:41:19 | INFO     | GeoOffice.services.database_service | База данных успешно инициализирована


Заполнение БД тестовыми данными

In [4]:
test_projects = [
    {
        'number': '201.17',
        'name': 'Инженерные сети и благоустройство жилого района усадебной застройки «Сосны» в г.Миоры',
        'customer': 'Дочернее коммунальное унитарное предприятие «Управление капитального строительства Глубокского района»',
        'chief_engineer': 'Ладисова Татьяна Геннадьевна',
        'status': 'активный',
        'address': 'Витебская область, Миорский район, г.Миоры',
        'path': '2025\\Миорский район\\201.17 Сосны, Миоры'
    },
    {
        'number': '92.25',
        'name': 'Возведение многоквартирного жилого дома по ул. Гагарина в г. Городоке',
        'customer': 'Государственное предприятие «УКС Городокского района»',
        'chief_engineer': 'Ладисова Татьяна Геннадьевна',
        'status': 'активный',
        'address': 'Витебская область, г. Городок, ул. Гагарина',
        'path': '"2025\\Городокский район\\92.25 ЖД Городок ул.Гагарина"'
    },
    ]

In [5]:
for test_project in test_projects:
    db_service.create_project(**test_project)

20:30:22 | DEBUG    | GeoOffice.services.database_service | Создан новый проект: Project(id=None, number='50.25', name='Реконструкция спортзала по адресу ул.Молодежная 26 в г.Новополоцк')
20:30:22 | DEBUG    | GeoOffice.services.database_service | Создан новый проект: Project(id=None, number='116.25', name='Ушачи вынос осей')
20:30:22 | DEBUG    | GeoOffice.services.database_service | Создан новый проект: Project(id=None, number='72.25', name='ЖД Шарковщина')


In [22]:
from datetime import datetime
from src.models.project_model import Project

project_data = db_service.get_project(1).to_dict()
project_data['created_date'] = datetime.fromisoformat(project_data['created_date'])
project_data['modified_date'] = datetime.fromisoformat(project_data['modified_date'])
Project(**project_data)

20:59:19 | DEBUG    | GeoOffice.services.database_service | Получение проекта по id: id=1


Project(id=1, number='50.25', name='Реконструкция спортзала по адресу ул.Молодежная 26 в г.Новополоцк', customer='Новополоцкий городской исполнительный комитет', chief_engineer='Богданова Екатерина Александровна', status='активный', address='ул.Молодежная 26 в г.Новополоцк', path='2025\\Новополоцк\\50.25 Реконструкция спортзала Новополоцк', created_date=datetime.datetime(2025, 7, 5, 20, 30, 22, 952641), modified_date=datetime.datetime(2025, 7, 5, 20, 30, 22, 952649))

In [20]:
db_service.models.Project.select()[:]

[ProjectTable[1], ProjectTable[2], ProjectTable[3]]